# Análisis Profundo del Proyecto CUVS: Introducción y Uso Básico

Este notebook proporciona una introducción profunda al proyecto CUVS de RAPIDS, una biblioteca para búsqueda de vectores y clustering acelerada por GPU. Cubriremos instalación, uso básico y pruebas de utilidad y rendimiento.

## 1. Instalar RAPIDS y CUVS

CUVS es parte del ecosistema RAPIDS, que acelera el análisis de datos con GPU NVIDIA.

### Requisitos
- GPU NVIDIA con CUDA 11.8+ (recomendado CUDA 12+)
- Python 3.8+
- Sistema operativo: Linux, Windows, macOS (con limitaciones)

### Instalación en Google Colab
En Colab, selecciona Runtime > Change runtime type > GPU (T4, V100, A100, etc.)

```bash
# Para CUDA 12 (recomendado)
!pip install cuvs-cu12 --extra-index-url=https://pypi.nvidia.com

# Para CUDA 11
# !pip install cuvs-cu11 --extra-index-url=https://pypi.nvidia.com
```

### Instalación Local
```bash
# conda (recomendado)
conda install -c rapidsai -c conda-forge cuvs

# pip
pip install cuvs-cu12 --extra-index-url=https://pypi.nvidia.com
```

### Verificación
Después de la instalación, verifica que CUVS esté disponible:

In [ ]:
# Instalar RAPIDS y CUVS
# Para CUDA 12 (común en Colab)
!pip install cuvs-cu12 --extra-index-url=https://pypi.nvidia.com

# O para CUDA 11
# !pip install cuvs-cu11 --extra-index-url=https://pypi.nvidia.com

# Verificar instalación
import cuvs
print("CUVS versión:", cuvs.__version__)

# Verificar GPU
import cupy as cp
print("GPU disponible:", cp.cuda.runtime.getDeviceCount() > 0)
if cp.cuda.runtime.getDeviceCount() > 0:
    print("GPU:", cp.cuda.runtime.getDeviceProperties(0)['name'].decode())

# Troubleshooting:
# - Si hay errores de CUDA, verifica la versión compatible
# - En macOS, CUVS tiene limitaciones; usa CPU alternatives como FAISS

## 2. Importar Librerías Requeridas

Importamos las librerías necesarias. CUVS usa CuPy para arrays GPU y NumPy para CPU.

### Librerías Principales
- `cuvs`: API principal de CUVS
- `cupy`: Arrays GPU (similar a NumPy)
- `numpy`: Arrays CPU
- `matplotlib`: Visualización de resultados

In [ ]:
import numpy as np
import cupy as cp
from cuvs.common import Resources
from cuvs.neighbors import ivf_flat
import time
import matplotlib.pyplot as plt

## 3. Cargar y Preparar Dataset

CUVS trabaja con vectores de punto flotante. Los datos deben estar en formato NumPy o CuPy arrays.

### Formatos Soportados
- **Float32/Float16**: Recomendado para precisión y velocidad
- **Dimensiones**: Típicamente 128-1024 para embeddings
- **Normalización**: Importante para métricas de similitud coseno

### Fuentes de Datos
- Embeddings de modelos como BERT, CLIP, Sentence Transformers
- Datos de ANN benchmarks (SIFT, GloVe)
- Vectores personalizados de tu aplicación

En este ejemplo usamos datos sintéticos similares a embeddings de texto/imágenes.

In [ ]:
# Generar datos sintéticos: 10000 vectores de 128 dimensiones
np.random.seed(42)
n_samples = 10000
dim = 128
dataset = np.random.randn(n_samples, dim).astype(np.float32)

# Normalizar para similitud coseno
dataset = dataset / np.linalg.norm(dataset, axis=1, keepdims=True)

# Queries: 100 vectores aleatorios
n_queries = 100
queries = np.random.randn(n_queries, dim).astype(np.float32)
queries = queries / np.linalg.norm(queries, axis=1, keepdims=True)

print(f"Dataset shape: {dataset.shape}")
print(f"Queries shape: {queries.shape}")

## 4. Construir Índice CUVS

Los índices CUVS permiten búsqueda rápida aproximada. IVF-Flat divide el espacio en clusters (listas) para búsqueda eficiente.

### Parámetros Clave
- `n_lists`: Número de clusters (más = mejor recall, pero más lento)
- `metric`: Distancia ("euclidean", "cosine", "inner_product")
- `add_data_on_build`: Construir y agregar datos en un paso

### Cuándo Usar Cada Algoritmo
- **IVF-Flat**: Datasets medianos (<1M vectores), balance velocidad/precisión
- **IVF-PQ**: Datasets grandes, optimización de memoria
- **CAGRA**: Búsqueda en tiempo real, máxima velocidad

In [ ]:
resources = Resources()

# Parámetros de construcción
build_params = ivf_flat.IndexParams(
    n_lists=1024,  # Número de clusters
    metric="cosine",  # Métrica de distancia
    add_data_on_build=True
)

# Construir índice
start_time = time.time()
index = ivf_flat.build(build_params, cp.asarray(dataset), resources=resources)
resources.sync()
build_time = time.time() - start_time

print(f"Índice construido en {build_time:.2f} segundos")
print(f"Índice: {index}")

## 5. Realizar Búsqueda de Vectores

La búsqueda encuentra los k vecinos más cercanos para cada query vector.

### Parámetros de Búsqueda
- `n_probes`: Clusters a buscar (más = mejor recall, más lento)
- `k`: Número de vecinos a retornar

### Rendimiento Esperado
- **IVF-Flat**: 1000-10000 QPS en GPU moderna
- **Latencia**: <1ms para k=10 en datasets medianos
- **Escalabilidad**: Lineal con tamaño del dataset

In [ ]:
# Parámetros de búsqueda
search_params = ivf_flat.SearchParams(n_probes=10)  # Número de clusters a buscar
k = 10  # Top-k vecinos

# Búsqueda
start_time = time.time()
distances, neighbors = ivf_flat.search(
    search_params, 
    index, 
    cp.asarray(queries), 
    k=k, 
    resources=resources
)
resources.sync()
search_time = time.time() - start_time

print(f"Búsqueda completada en {search_time:.4f} segundos")
print(f"Tiempo por query: {search_time / n_queries * 1000:.2f} ms")
print(f"Distances shape: {distances.shape}")
print(f"Neighbors shape: {neighbors.shape}")

## 6. Evaluar Precisión de Búsqueda

Medimos qué tan buenos son los resultados aproximados vs exactos.

### Métricas Principales
- **Recall@k**: Fracción de resultados correctos en top-k
- **Precision@k**: Exactitud en top-k
- **Mean Reciprocal Rank (MRR)**: Posición promedio del primer resultado correcto

### Interpretación
- Recall > 0.9: Excelente para la mayoría de aplicaciones
- Recall > 0.8: Bueno para búsqueda general
- Recall < 0.7: Puede requerir refinamiento o algoritmo diferente

In [ ]:
# Calcular distancias exactas usando brute force
from sklearn.metrics.pairwise import cosine_distances
exact_distances = cosine_distances(queries, dataset)
exact_neighbors = np.argsort(exact_distances, axis=1)[:, :k]

# Calcular recall
def recall_at_k(pred, true, k):
    recall = 0
    for i in range(len(pred)):
        recall += len(set(pred[i]) & set(true[i])) / k
    return recall / len(pred)

approx_neighbors = cp.asnumpy(neighbors)
recall = recall_at_k(approx_neighbors, exact_neighbors, k)
print(f"Recall@{k}: {recall:.4f}")

## 7. Benchmark de Métricas de Rendimiento

Medimos velocidad y eficiencia del sistema.

### Métricas de Rendimiento
- **QPS (Queries Per Second)**: Throughput del sistema
- **Latencia**: Tiempo por query individual
- **Uso de Memoria**: RAM/GPU utilizada
- **Tiempo de Construcción**: Costo inicial del índice

### Factores que Afectan el Rendimiento
- **Tamaño del Dataset**: Más grande = más lento (pero escalable)
- **Dimensionalidad**: Más dimensiones = más costoso
- **Parámetros del Índice**: Trade-off recall vs velocidad
- **Hardware**: GPU más nueva/mejor = mejor rendimiento

In [ ]:
print(f"Tiempo de construcción del índice: {build_time:.2f} s")
print(f"Tiempo total de búsqueda: {search_time:.4f} s")
print(f"Queries por segundo (QPS): {n_queries / search_time:.2f}")

# Uso de memoria (aproximado)
import psutil
process = psutil.Process()
mem_usage = process.memory_info().rss / (1024 ** 3)  # GB
print(f"Uso de memoria aproximado: {mem_usage:.2f} GB")

## 8. Comparar con Alternativas Basadas en CPU

Comparamos CUVS GPU con FAISS CPU para mostrar la aceleración.

### Alternativas CPU
- **FAISS**: Librería ANN de Facebook, excelente baseline
- **Annoy**: Basado en árboles, simple pero limitado
- **HNSW**: Algoritmo gráfico, buena precisión pero lento en construcción

### Cuándo Usar GPU vs CPU
- **GPU (CUVS)**: Datasets >100K vectores, búsqueda en tiempo real, alta throughput
- **CPU (FAISS)**: Desarrollo local, datasets pequeños, sin GPU disponible

### Aceleración Esperada
- **10-100x** más rápido en búsqueda
- **5-20x** más rápido en construcción de índices
- **Mejor escalabilidad** para datasets grandes

In [ ]:
# Instalar FAISS si no está
!pip install faiss-cpu

import faiss

# Construir índice FAISS
index_faiss = faiss.IndexFlatIP(dim)  # Producto interno para coseno
index_faiss.add(dataset)

# Búsqueda FAISS
start_time_faiss = time.time()
distances_faiss, neighbors_faiss = index_faiss.search(queries, k)
search_time_faiss = time.time() - start_time_faiss

print(f"FAISS búsqueda tiempo: {search_time_faiss:.4f} s")
print(f"CUVS búsqueda tiempo: {search_time:.4f} s")
print(f"Aceleración: {search_time_faiss / search_time:.2f}x")

# Recall FAISS (exacto)
recall_faiss = recall_at_k(neighbors_faiss, exact_neighbors, k)
print(f"Recall FAISS: {recall_faiss:.4f}")
print(f"Recall CUVS: {recall:.4f}")

## 9. Optimizar Parámetros del Índice

El rendimiento depende de parámetros bien ajustados.

### Guía de Optimización
- **n_lists**: sqrt(n_samples) como punto de partida
- **n_probes**: 1-10% de n_lists para balance velocidad/precisión
- **k**: 10-100 típico para aplicaciones de búsqueda

### Estrategia de Tuning
1. Fijar n_lists basado en tamaño del dataset
2. Variar n_probes para encontrar trade-off óptimo
3. Medir recall y QPS para tu caso de uso específico
4. Considerar refinamiento si recall es insuficiente

In [ ]:
# Experimentar con n_probes
n_probes_list = [1, 5, 10, 20, 50]
recalls = []
times = []

for n_probes in n_probes_list:
    search_params = ivf_flat.SearchParams(n_probes=n_probes)
    start = time.time()
    dist, neigh = ivf_flat.search(search_params, index, cp.asarray(queries), k=k, resources=resources)
    resources.sync()
    t = time.time() - start
    r = recall_at_k(cp.asnumpy(neigh), exact_neighbors, k)
    recalls.append(r)
    times.append(t)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(n_probes_list, recalls, marker='o')
plt.xlabel('n_probes')
plt.ylabel('Recall@10')
plt.title('Recall vs n_probes')

plt.subplot(1, 2, 2)
plt.plot(n_probes_list, times, marker='o')
plt.xlabel('n_probes')
plt.ylabel('Tiempo de búsqueda (s)')
plt.title('Tiempo vs n_probes')
plt.show()

## 10. Manejar Datasets a Gran Escala

CUVS está diseñado para datasets masivos.

### Estrategias de Escalado
- **Procesamiento por lotes**: Construir índices en chunks
- **Multi-GPU**: Distribuir carga entre múltiples GPUs
- **Compresión**: Usar IVF-PQ para reducir memoria
- **Optimización de memoria**: RMM pool allocator

### Límites Prácticos
- **Millones de vectores**: Fácil con configuración adecuada
- **Miles de millones**: Posible con multi-GPU y optimizaciones
- **Memoria**: Monitorear uso GPU/CPU, ajustar parámetros según necesidad

In [ ]:
# Para datasets grandes, procesar en lotes
large_dataset = np.random.randn(50000, dim).astype(np.float32)
large_dataset = large_dataset / np.linalg.norm(large_dataset, axis=1, keepdims=True)

batch_size = 10000
for i in range(0, len(large_dataset), batch_size):
    batch = large_dataset[i:i+batch_size]
    # Extender índice con batch
    ivf_flat.extend(index, cp.asarray(batch), cp.arange(i, i+len(batch), dtype=cp.int64))
    resources.sync()

print("Índice extendido con datos adicionales")

# Búsqueda en índice grande
distances_large, neighbors_large = ivf_flat.search(
    search_params, index, cp.asarray(queries), k=k, resources=resources
)
resources.sync()
print(f"Búsqueda en dataset grande completada")